# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [2]:
## the dataset link
# https://jdgrossman.com/assets/spam.csv

import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("https://jdgrossman.com/assets/spam.csv")
df.head()


,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
0,0.00,0.64,0.64,0.0,0.32,0.00,0.00,0.00,0.00,0.00,...,0.00,0.000,0.0,0.778,0.000,0.000,3.756,61,278,1
1,0.21,0.28,0.50,0.0,0.14,0.28,0.21,0.07,0.00,0.94,...,0.00,0.132,0.0,0.372,0.180,0.048,5.114,101,1028,1
2,0.06,0.00,0.71,0.0,1.23,0.19,0.19,0.12,0.64,0.25,...,0.01,0.143,0.0,0.276,0.184,0.010,9.821,485,2259,1
3,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.137,0.0,0.137,0.000,0.000,3.537,40,191,1
4,0.00,0.00,0.00,0.0,0.63,0.00,0.31,0.63,0.31,0.63,...,0.00,0.135,0.0,0.135,0.000,0.000,3.537,40,191,1


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [3]:
num_emails = len(df)
num_emails


4601

### What fraction of the emails are spam?

In [4]:
spam_fraction = df['is_spam'].mean()
spam_fraction


np.float64(0.39404477287546186)

### Which email has the highest % of words matching "money"?

In [5]:
idx = df['money'].idxmax()
highest_money_email = df.loc[idx]
highest_money_email['money'], idx


(np.float64(12.5), 545)

## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [6]:
X = df[['char_dollar', 'credit', 'money', 're']]
X = sm.add_constant(X)
y = df['is_spam']

lpm = sm.OLS(y, X).fit()
print(lpm.summary())


                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        06:23:20   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [7]:
pred_probs = lpm.predict(X)
pred_min = pred_probs.min()
pred_max = pred_probs.max()
pred_min, pred_max


(-0.8125403869560923, 3.8494089964108236)

LPM predictions can be < 0 or > 1, which is impossible for true probabilities.

This is one reason logistic regression is preferred.

## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [8]:
def prob_to_odds(p):
    return p / (1 - p)

def odds_to_prob(o):
    return o / (1 + o)

# Tests
test1 = odds_to_prob(2)      # 2:1 odds → 2/3
test2 = prob_to_odds(2/3)    # 2/3 prob → odds = 2
test1, test2


(0.6666666666666666, 1.9999999999999998)

### If my winning probability is 60%, and I double my odds, what is my new probability?

In [9]:
p = 0.60
o = prob_to_odds(p)
new_o = 2 * o
new_p = odds_to_prob(new_o)
new_p


0.75

## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [11]:
import statsmodels.formula.api as smf

logit_model = smf.glm(
    "is_spam ~ char_dollar + credit + money + re",
    data=df,
    family=sm.families.Binomial()
).fit()

print(logit_model.summary())


                 Generalized Linear Model Regression Results                  
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                            GLM   Df Residuals:                     4596
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2213.9
Date:                Tue, 18 Nov 2025   Deviance:                       4427.8
Time:                        06:30:35   Pearson chi2:                 1.00e+10
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3152
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.0666      0.043    -24.680      

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [12]:
b = logit_model.params
b


,0
Intercept,-1.066563
char_dollar,11.817567
credit,2.311898
money,1.993280
re,-0.775505
